# Пакет bt для тестирования торговых систем на языке Python

Проверялось в Python 2.7.

## Установка пакета bt

Для установки пакета `bt` в системе Linux/Ubuntu достаточно выполнить в командной строке::

    sudo pip install bt

Для системы Windows ``sudo`` опускаем.

Если лень открывать терминал с командной строкой, то можно выполнить установку прямо на странице IPython Notebook::

    !pip install bt
	
Вместе с пакетом `bt` будет установлен также `ffn`.

# Для установки пакета:
!pip install bt

In [ ]:
# Чтобы графики выводились внутри страницы:
%matplotlib inline

## Загрузка котировок

В пакете `bt` имеется собственная функция для загрузки котировок с сайта Yahoo Finance. Для примера скачаем цены закрытия (adjusted close) двух биржевых фондов (ETF): `GDX` и `SPY`: 

In [ ]:
import bt
data = bt.get('snp', start='2016-01-01')
data

In [ ]:
print (data.head(3)) # Вывели первые 3 записи для проверки.

## Класс Strategy

Для тестирования торговой системы надо сначала создать экземпляр класса `Strategy`, в котором задать нужную комбинацию алгоритмов ([`algos`](http://pmorissette.github.io/bt/algos.html)). Алгоритмы указывают, как выбирать бумаги для покупки и продажи, и в каких пропорциях распределять имеющиеся средства между бумагами (т.е. способ вычисления весовых коэффициентов).

In [ ]:
s1 = bt.Strategy('s1', [bt.algos.RunMonthly(),  # Выполняем ежемесячно;
                       bt.algos.SelectAll(),    # выбираем все бумаги;
                       bt.algos.WeighEqually(), # в равной пропорции;
                       bt.algos.Rebalance()])   # выполняем ребалансировку согласно выбранной пропорции.

Здесь мы указали, что будем проводить ежемесячную (`RunMonthly`) ребалансировку (`Rebalance`) портфеля, распределяя все имеющиеся средства поровну (`WeighEqually`) между всеми выбранными бумагами (`SelectAll`).


Усложним стратегию. Будем проводить еженедельную ребалансировку (`RunWeekly`), а весовые коэффициенты для каждой бумаги рассчитывать обратно пропорционально волатильности (`WeighInvVol`):

In [ ]:
s2 = bt.Strategy('s2', [bt.algos.RunWeekly(),   # Еженедельно;
                        bt.algos.SelectAll(),   # выбираем все бумаги;
                        bt.algos.WeighInvVol(), # обратно пропорционально волатильности;
                        bt.algos.Rebalance()])  # выполняем ребалансировку согласно весовым коэффициентам.

Теперь проверим классическую стратегию торговли по скользящей средней: 
будем покупать только те бумаги, которые выше своей 250-дневной скользящей средней. 
Для выбора бумаг по заданному условию используем встроенный алгоритм `SelectWhere`:

In [ ]:
import pandas as pd
#sma1 = pd.rolling_mean(data, 250) # Простая скользящая средняя.
sma1 = pd.rolling(data, 250).mean()
s3 = bt.Strategy('SMA1', [ 
         bt.algos.SelectWhere(data > sma1), # Выбираем только те бумаги, которые выше скользящей средней;
         bt.algos.WeighEqually(),           # делим средства в равной попорции.  
         bt.algos.Rebalance()] )

Результаты торговли по любой системе обычно сравнивают с каким-нибудь *бенчмарком*. Часто в качестве бенчмарка выбирают стратегию "купи и держи" (buy & hold):

In [ ]:
s0 = bt.Strategy('bh', [bt.algos.RunOnce(),       # Только один раз;
                        bt.algos.SelectAll(),     # выбираем все бумаги;
                        bt.algos.WeighEqually(),  # в равной пропорции.
                        bt.algos.Rebalance()])

Итак, мы определили четыре торговые стратегии. Пора их протестировать.

## Тестирование торговых стратегий

Перед запуском теста надо создать экземпляр класса `Backtest`, 
указав стратегию и данные для тестирования, после чего можно запустить тест на выполнение:

In [ ]:
test1 = bt.Backtest(s1, data)
res1 = bt.run(test1)

Результат можно отобразить в виде графика эквити:

In [ ]:
res1.plot()

Таблица результатов:

In [ ]:
res1.display()

Гистограмма распределения прибылей/убытков (return):

In [ ]:
res1.plot_histogram()

Весовые коэффициенты для каждой бумаги:

In [ ]:
res1.plot_security_weights()

Можно запустить тест сразу для нескольких стратегий:

In [ ]:
test0 = bt.Backtest(s0, data)
test1 = bt.Backtest(s1, data)
test2 = bt.Backtest(s2, data)
test3 = bt.Backtest(s3, data)
res = bt.run(test1, test2, test3, test0) # Запустили четыре теста для последующего сравнения.
res.plot()

Сравним результаты тестирования всех четырёх стратегий: 

In [ ]:
res.display()

## Построение графиков

Отобразим на графике скользящую среднюю:

In [ ]:
plot = bt.merge(data, sma1).plot(figsize=(17, 6)).legend(['GDX','SPY','GDX SMA', 'SPY SMA'], loc='upper left')

При построении графика можно указать имя какой-то одной стратегии:

In [ ]:
res.plot_security_weights('SMA1')

## Оптимизация

Теперь проверим, какие будут результаты при различных значениях периода скользящей средней. Для этого оформим код стратегии в виде функции:

In [ ]:
def above_sma(tickers, period=250, start='2010-01-01', name='above_sma'):
    """
    Покупаем бумагу, если она выше скользящей средней.
    """
    data = bt.get(tickers, start=start)
    sma1 = pd.rolling_mean(data, period)
    s = bt.Strategy(name, [bt.algos.SelectWhere(data > sma1),
                           bt.algos.WeighEqually(),
                           bt.algos.Rebalance()])
    return bt.Backtest(s, data)

Также определим отдельную функцию для *бенчмарка* -- 
стратегии "купи и держи" (buy & hold), с которой будем сравнивать результаты всех остальных стратегий:

In [ ]:
def bh(tickers, start='2010-01-01', name='buy&hold'):
    s = bt.Strategy(name, [bt.algos.RunOnce(),       # Только один раз;
                           bt.algos.SelectAll(),     # выбираем все бумаги;
                           bt.algos.WeighEqually(),  # в равной пропорции.
                           bt.algos.Rebalance()])
    data = bt.get(tickers, start=start)  # Получаем котировки.
    return bt.Backtest(s, data)          # Запускаем тестирование и возвращаем результат.

Всё готово для того, чтобы задать бумаги для торговли и выполнить тестирование для разных значений периода скользящей средней:

In [ ]:
# Бумаги для торговли:
tickers = 'aapl,msft,c,gs,ge'

# Тесты с разными значениями периода скользящей средней:
sma1 = above_sma(tickers, period=10, name='sma10')
sma2 = above_sma(tickers, period=20, name='sma20')
sma3 = above_sma(tickers, period=50, name='sma50')
sma4 = above_sma(tickers, period=100, name='sma100')
sma5 = above_sma(tickers, period=150, name='sma150')
sma6 = above_sma(tickers, period=200, name='sma200')

# Бенчмарк:
benchmark = bh('spy', name='bh_spy')

# Выполняем все тесты:
res = bt.run(sma1, sma2, sma3, sma4, sma5, sma6, benchmark)

In [ ]:
res.plot()

In [ ]:
res.display()

Теперь мы можем выбрать период, при котором достигается наибольшее значение итоговой прибыли или, например, коэффициента Шарпа.

## Пересечение двух скользящих средних

Наконец, протестируем классическую торговую систему с двумя скользящими средними, имеющими разный период. Будем покупать, когда быстрая скользящая средняя пересекает медленную снизу вверх, и продавать при обратном пересечении.

In [ ]:
data = bt.get('spy', start='2010-01-01')
sma1 = pd.rolling_mean(data, 50)   # Быстрая скользящая средняя.
sma2 = pd.rolling_mean(data, 150)  # Медленная скользящая средняя.
tw = sma2.copy()             # Создали новую колонку копированием.
tw[sma1 > sma2] = 1.0        # Весовой коэффициент для покупки.
tw[sma1 <= sma2] = -1.0      # Весовой коэффициент для короткой продажи.
tw[sma2.isnull()] = 0.0  # Первые значения, когда скользящая средняя не определена.

In [ ]:
tmp = bt.merge(tw, data, sma1, sma2)                # Соединили колонки.
tmp.columns = ['tw', 'price', 'sma1', 'sma2']       # Имена колонок.
ax = tmp.plot(figsize=(15,5), secondary_y=['tw'])   # График.

Значения из колонки весовых коэффициентов `tw` отображены на графике синей линией; соответствующая шкала для вертикальной оси расположена справа. Шкала для остальных значений (цены и скользящих средних) нанесена слева.

In [ ]:
ma_cross = bt.Strategy('ma_cross', [bt.algos.WeighTarget(tw),
                                    bt.algos.Rebalance()])
t = bt.Backtest(ma_cross, data)
res = bt.run(t)

In [ ]:
res.plot()

In [ ]:
res.display()

См. http://profitraders.com/Python/btIntro.html